# Integrated Simulations

In [ ]:
import os; from os import listdir; from os.path import isfile, join
import re  
from skimage.io import imread
from skimage.exposure import rescale_intensity
import numpy as np 
from tqdm.notebook import tqdm
from timeit import default_timer as timer
import scipy
import pandas as pd
import shutil
import pathlib
import sys
import seaborn as sns
import rsnapsim as rss
import scipy.stats as stats
import matplotlib.pyplot as plt 
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.patches import Rectangle
from matplotlib.patches import ConnectionPatch


In [ ]:
plt.style.use("dark_background")

In [ ]:
# Defining directories
current_dir = pathlib.Path().absolute()
sequences_dir = current_dir.parents[1].joinpath('DataBases','gene_files')
video_dir = current_dir.parents[1].joinpath('DataBases','videos_for_sim_cell')
rsnaped_dir = current_dir.parents[1].joinpath('rsnaped')
gene_file = current_dir.parents[1].joinpath('DataBases','gene_files','KDM5B_withTags.txt')
masks_dir = current_dir.parents[1].joinpath('DataBases','masks_for_sim_cell')

In [ ]:
sequences_dir

In [ ]:
# Importing rSNAPed
sys.path.append(str(rsnaped_dir))
import rsnaped as rsp

In [ ]:
rsp.Banner().print_banner()

In [ ]:
# These are the parameters that need to be tested. 
number_of_simulated_cells = 1     # PLEASE TEST MIN 1 MAX 10
number_spots_per_cell = 60           # PLEASE TEST MIN 5 MAX 200
simulation_time_in_sec = 500          # PLEASE TEST MIN 10 MAX 100
step_size_in_sec = 1

min_percentage_time_tracking = 0.3   # (normalized) minimum time to consider a trajectory.
average_cell_diameter = 400

In [ ]:
diffusion_coefficient = 0.05       # This value must have units of micrometers_square_per_seconds
microns_per_pixel = 0.13

In [ ]:
intensity_calculation_method = 'gaussian_fit'  # options are : 'total_intensity' and 'disk_donut' 'gaussian_fit'
mask_selection_method = 'max_area' # options are : 'max_spots' and 'max_area' 
use_optimization_for_tracking = 1 # 0 not using, 1 is using optimization

selected_channel_tracking = 0
selected_channel_segmentation = 1

frame_selection_empty_video = 'gaussian' # Options are: 'constant' , 'shuffle' and 'loop' 'linear_interpolation', 'gaussian', 'poisson'
dataframe_format = 'long' # 'short'  'long'

store_videos_in_memory = False
save_as_gif = False 
show_plot = False  # Flag to show plots for the detection and tracking process.


In [ ]:
spot_size = 5 # spot size for the simulation and tracking.
spot_sigma = 1
elongation_rate = 10
initiation_rate = 0.03
simulated_RNA_intensities_method = 'random'

In [ ]:
basal_intensity_in_background_video = 10000
scale_intensity_in_base_video=False

In [ ]:
intensity_scale_ch0 = 1
intensity_scale_ch1 = 1
intensity_scale_ch2 = None

In [ ]:
particle_detection_size = spot_size

In [ ]:
select_background_cell_index = 0 # Integer in range 0 to 8, or use None to select a random value. 
perform_video_augmentation = False 

## Running the simulations

In [ ]:
list_videos, list_dataframe_simulated_cell, merged_dataframe_simulated_cells, ssa_trajectories, list_files_names, video_path, dataframe_path = rsp.simulate_cell( video_dir, 
                                                                        list_gene_sequences = gene_file,
                                                                        list_number_spots= number_spots_per_cell,
                                                                        list_target_channels_proteins = 1,
                                                                        list_target_channels_mRNA = 0, 
                                                                        list_diffusion_coefficients=diffusion_coefficient,
                                                                        list_elongation_rates=elongation_rate,
                                                                        list_initiation_rates=initiation_rate,
                                                                        masks_dir=masks_dir, 
                                                                        list_label_names=1,
                                                                        number_cells = number_of_simulated_cells,
                                                                        simulation_time_in_sec = simulation_time_in_sec,
                                                                        step_size_in_sec = step_size_in_sec,
                                                                        save_as_gif = save_as_gif,
                                                                        frame_selection_empty_video=frame_selection_empty_video,
                                                                        spot_size = spot_size,
                                                                        spot_sigma = spot_sigma,
                                                                        intensity_scale_ch0 = intensity_scale_ch0,
                                                                        intensity_scale_ch1 = intensity_scale_ch1,
                                                                        intensity_scale_ch2 = intensity_scale_ch2,
                                                                        dataframe_format = 'long',
                                                                        simulated_RNA_intensities_method=simulated_RNA_intensities_method,
                                                                        store_videos_in_memory= store_videos_in_memory,
                                                                        scale_intensity_in_base_video=scale_intensity_in_base_video,
                                                                        basal_intensity_in_background_video=basal_intensity_in_background_video,
                                                                        microns_per_pixel=microns_per_pixel,
                                                                        select_background_cell_index=select_background_cell_index,
                                                                        perform_video_augmentation=perform_video_augmentation)

In [ ]:
list_dataframe_simulated_cell[0]

In [ ]:
number_images = len(list_videos)

In [ ]:
# # Reads the folder with the results and import the simulations as lists
list_files_names = sorted([f for f in listdir(video_path) if isfile(join(video_path, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
path_files = [ str(video_path.joinpath(f).resolve()) for f in list_files_names ] # creating the complete path for each file
path_files.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order

# # Reading the microscopy data
number_images = number_of_simulated_cells
number_images

# Display simulations for single time point

## <span style="color:lightgreen">Channel 1</span>

In [ ]:
# Showing the simulated images
list_videos = [imread(f)[:,:,:,:] for f in  path_files] # List with all the videos
rsp.VisualizerImage(list_videos,list_files_names=list_files_names,selected_channel =1,selected_time_point= 0,normalize=0,individual_figure_size=7).plot()
del list_videos

In [ ]:
list_DataFrame_particles_intensities, list_array_intensities, list_time_vector, list_selected_mask = rsp.image_processing( files_dir_path_processing=video_path,
                                                                                                                            particle_size=particle_detection_size,
                                                                                                                            selected_channel_tracking = selected_channel_tracking,
                                                                                                                            selected_channel_segmentation = selected_channel_segmentation,
                                                                                                                            intensity_calculation_method =intensity_calculation_method, 
                                                                                                                            mask_selection_method = mask_selection_method,
                                                                                                                            show_plot=show_plot,
                                                                                                                            use_optimization_for_tracking=use_optimization_for_tracking,
                                                                                                                            real_positions_dataframe = list_dataframe_simulated_cell,
                                                                                                                            average_cell_diameter=average_cell_diameter,
                                                                                                                            print_process_times=True,
                                                                                                                            min_percentage_time_tracking=min_percentage_time_tracking,
                                                                                                                            dataframe_format=dataframe_format)

In [ ]:
list_DataFrame_particles_intensities[0].head()

In [ ]:
selected_channel = 1

In [ ]:
# Calculate background intensity distribution.
list_videos = [imread(f)[:,:,:,:] for f in  path_files] # List with all the videos
image = list_videos[0]
df_sim = list_dataframe_simulated_cell[0].copy()
print(image.shape)
del list_videos

In [ ]:
# Apply mask to image.
selected_mask= list_selected_mask[0].copy()
img_removed_mask = rsp.MaskingImage(video=image,mask=selected_mask).apply_mask()
# Returns the pixel values for the image, removing the positions where spots are detected.
test_image = img_removed_mask[:,:,:,selected_channel].copy()
for i in range (test_image.shape[0]):
    x_values = df_sim.loc[(df_sim['frame']==i)]['x'].values
    y_values = df_sim.loc[(df_sim['frame']==i)]['y'].values
    test_image[i]=rsp.Utilities.remove_spots_from_image(test_image[i], x_values, y_values,spot_size)
plt.imshow(test_image[10])

In [ ]:
sim_wo_spots = test_image[test_image>0]
sim_w_spots = img_removed_mask[:,:,:,selected_channel].flatten()
sim_w_spots = sim_w_spots[sim_w_spots>0]
sim_wo_spots = sim_wo_spots[sim_wo_spots<20000]
sim_w_spots = sim_w_spots[sim_w_spots<20000]

In [ ]:
# section that detects pixels without zeros for all frames
spot_count = 0
number_background_pixels =100
background_pixel_trajectories = np.zeros((number_background_pixels, test_image.shape[0])  )
while spot_count < number_background_pixels:
    test_pixel = np.random.randint(low=0, high=test_image.shape[1], size=(2), dtype=int)
    temp = test_image[:,test_pixel[0],test_pixel[1]].flatten()
    if np.all(temp): # test if the pixel has any zeros
        background_pixel_trajectories [spot_count,:] = temp
        spot_count+=1
    

In [ ]:
plt.figure(figsize=(5,5))
plt.style.use(['default', 'fivethirtyeight'])
plt.plot(background_pixel_trajectories.T)
plt.show

In [ ]:
mean_acf_bg_sim, err_acf_bg_sim, lags_sim, decorrelation_time_bg_sim, auto_correlation_matrix_bg_sim = rsp.Covariance(intensity_array=background_pixel_trajectories, max_lagtime= 40, show_plot= True,figure_size=(6,4)).calculate_autocovariance()

In [ ]:
# Pixel intensity distribution for the background
plt.style.use(['default', 'fivethirtyeight'])
plt.figure(figsize=(7,5))
plt.hist(sim_w_spots, bins=50, alpha=0.5, label="with_spots", stacked=True, histtype='barstacked',edgecolor='orangered',linewidth=1)
plt.hist(sim_wo_spots, bins=50, alpha=0.8, label="background", stacked=True, histtype='barstacked',edgecolor='k',linewidth=1)
plt.xlabel("Intensity", size=14)
plt.ylabel("Count", size=14)
plt.title("Pixel intensity distribution")
plt.legend(loc='upper right')
#plt.xlim((0,10000))
plt.show()

In [ ]:
#df = list_DataFrame_particles_intensities[0]
#snr_green_channel = rsp.Utilities.extract_field_from_dataframe( dataframe = df_sim, selected_time = 0, selected_field = 'SNR_green')
snr_green_channel_sim = rsp.Utilities.extract_field_from_dataframe( dataframe = df_sim, selected_time = None, selected_field = 'SNR_green').flatten()

# Dataframe with fields [image_number, cell_number, particle, frame, red_int_mean, green_int_mean, blue_int_mean, red_int_std, green_int_std, blue_int_std, x, y, SNR_red,SNR_green,SNR_blue].
plt.figure(figsize=(7,5))
plt.hist(snr_green_channel_sim, bins=50,histtype='barstacked',edgecolor='orangered',linewidth=2,color = '#1C00FE', alpha=0.8)
plt.xlabel("SNR", size=14)
plt.ylabel("Count", size=14)
plt.title("SNR distribution")
#plt.legend(loc='upper right')
#plt.xlim((0,35000))
plt.show()
print('SNR mean =', np.round(np. median(snr_green_channel_sim)))

In [ ]:
int_green_channel_sim = rsp.Utilities.extract_field_from_dataframe( dataframe = df_sim, selected_time = None, selected_field = 'green_int_mean').flatten()
# Dataframe with fields [image_number, cell_number, particle, frame, red_int_mean, green_int_mean, blue_int_mean, red_int_std, green_int_std, blue_int_std, x, y, SNR_red,SNR_green,SNR_blue].
plt.figure(figsize=(7,5))
plt.hist(int_green_channel_sim, bins=30,histtype='barstacked',edgecolor='orangered',linewidth=2,color = '#1C00FE', alpha=0.8)
plt.xlabel("Intensity", size=14)
plt.ylabel("Count", size=14)
plt.title("Spot intensity distribution")
#plt.legend(loc='upper right')
#plt.xlim((0,35000))
plt.show()

In [ ]:
mean_acf_data,err_acf_data,lags, decorrelation_time, auto_correlation_matrix=rsp.Covariance(dataframe_particles=df_sim,selected_field='green_int_mean', max_lagtime= 400, show_plot= True,figure_size=(6,4)).calculate_autocovariance()

In [ ]:
calculated_diffusion_coefficient, MSD_series, trackpy_df = rsp.ParticleMotion(trackpy_dataframe=df_sim,microns_per_pixel=microns_per_pixel,step_size_in_sec=1.,max_lagtime=50,show_plot=True,remove_drift=False).calculate_msd()

In [ ]:
np.round(calculated_diffusion_coefficient,3)

# Real data

In [ ]:
# real video path
real_video_path = current_dir.parents[1].joinpath('DataBases','KDM5B_Translation_SampleMovies')

# Reads the folder with the results and import the simulations as lists
list_files_names = sorted([f for f in listdir(real_video_path) if isfile(join(real_video_path, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
path_files = [ str(real_video_path.joinpath(f).resolve()) for f in list_files_names ] # creating the complete path for each file
path_files.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
print(path_files)
# # Reading the microscopy data
number_images_real = len(path_files)
number_images_real

In [ ]:
# converting directory of videos to standard format
import tifffile
new_video_path = current_dir.parents[1].joinpath('DataBases','KDM5B_translation_samplemovies_correct_format')
# Showing the simulated images
for i in range (4):
    list_videos_real = [imread(f)[:,:,:,:] for f in  path_files] # List with all the videos
    real_video = rsp.ConvertToStandardFormat(video=list_videos_real[i], time_position = 0, height_position = 2,  width_position = 3, channel_position = 1 ).transpose_video()
    tifffile.imwrite(str(new_video_path.joinpath(pathlib.Path(path_files[i]).stem+'tif')), real_video)



In [ ]:
# Showing the simulated images

selected_real_video = 3
list_videos_real = [imread(f)[:,:,:,:] for f in  path_files] # List with all the videos
real_video = rsp.ConvertToStandardFormat(video=list_videos_real[selected_real_video], time_position = 0, height_position = 2,  width_position = 3, channel_position = 1 ).transpose_video()

pathlib.Path(path_files[0]).name
tifffile.imwrite(str(save_to_path_video.joinpath(saved_file_name+'.tif')), video)

print('Shape of real video', real_video.shape)
rsp.VisualizerImage(real_video,list_files_names=list_files_names,selected_channel =1,selected_time_point= 0,normalize=0,individual_figure_size=7).plot()
del list_videos_real

In [ ]:
# use pipeline to segment and detect spots
list_DataFrame_particles_intensities, list_array_intensities, list_time_vector, list_selected_mask = rsp.image_processing( files_dir_path_processing=video_path,
                                                                                                                            particle_size=particle_detection_size,
                                                                                                                            selected_channel_tracking = selected_channel_tracking,
                                                                                                                            selected_channel_segmentation = selected_channel_segmentation,
                                                                                                                            intensity_calculation_method =intensity_calculation_method, 
                                                                                                                            mask_selection_method = mask_selection_method,
                                                                                                                            show_plot=show_plot,
                                                                                                                            use_optimization_for_tracking=use_optimization_for_tracking,
                                                                                                                            real_positions_dataframe = list_dataframe_simulated_cell,
                                                                                                                            average_cell_diameter=average_cell_diameter,
                                                                                                                            print_process_times=True,
                                                                                                                            min_percentage_time_tracking=min_percentage_time_tracking,
                                                                                                                            dataframe_format=dataframe_format)

In [ ]:
# Remove spots to calculate background intensity

In [ ]:
# Autocorrelation function for background intensity

In [ ]:
# Intensity distribution for spots

In [ ]:
# 